# Các mẫu thiết kế: Template Method

**Bs. Lê Ngọc Khả Nhi**

# Giới thiệu

Khi viết code để làm phân tích dữ liệu, chắc các bạn cũng nhận ra rằng một số phân tích đều áp dụng một quy trình tương tự nhau, thí dụ phân tích hồi quy đa biến cho nghiên cứu diễn dịch thường có 3 bước: Centering dữ liệu đầu vào để trung bình dịch chuyển về 0; sau đó khớp một mô hình; sau cùng là báo cáo nội dung mô hình. Tuy nhiên khi áp dụng cho các loại mô hình khác nhau, thí dụ Logistic, Gaussian, Gamma..., nội dung mỗi quy trình cần được thay đổi chút ít cho phù hợp. Thí dụ với hồi quy Gamma, ta không thể centering biến kết quả vì kết quả phải là giá trị dương; tương tự, với hồi quy logistic khi báo cáo kết quả ta cần tính thêm Odds-ratio...

Trong bài này, Nhi sẽ giới thiệu về một mẫu thiết kế OOP rất dơn giản và tiện lợi cho quy trình như vậy, đó là Template Method.

**Template Method** là một mẫu thiết kế thuộc nhóm “Hành vi”, với cơ chế là sử dụng một superclass để triển khai một quy trình gồm nhiều công đoạn dưới dạng methods, nhưng cho phép những subclass thay đổi hành vi một phần các methods này để thích nghi với nhiều hoàn cảnh áp dụng khác nhau.

Template Method là một mẫu thiết kế thông dụng trong ngôn ngữ Python. Đây là mô thức được ưa chuộng bởi nhiều lập trình viên khi phải xây dựng một quy trình mà sau đó sẽ được áp dụng trong nhiều hoàn cảnh khác nhau theo cùng trình tự, nhưng mỗi lần như vậy cần thay đổi vài công đoạn để tương thích với hoàn cảnh mới. Sự thông dụng của Template Method còn vì nó rất dễ triển khai (chỉ dùng tính năng inheritance). 

# Cách triển khai

Đầu tiên, ta phân tích nội dung của quy trình cần thực thi, xem có khả năng phân rã nó ra thành từng mảnh nhỏ riêng biệt và theo trình tự hay không ? Trong đó, những công đoạn nào là chung cho mọi hoàn cảnh áp dụng nhưng yêu cầu ít nhiều thay đổi để thích nghi trong từng hoàn cảnh ?

Dựa vào danh sách công đoạn này, ta thiết kế một abstract base class gồm những abstract methods tương ứng cho từng công đoạn cần được thay đổi trong quy trình. Ta cũng có quyền tạo những method không thuộc loại abstract method, những method này sẽ được kế thừa chung và bất biến cho mọi subclass.

Tiếp theo, tương ứng với mỗi hoàn cảnh áp dụng khác nhau, ta thiết kế cho nó 1 concrete subclass, kế thừa tất cả methods (bao gồm abstract method là các quy trình có thể thay đổi được) từ abstract class, sau đó tùy biến nội dung bên trong một vài methods để thích nghi cho hoàn cảnh chuyên biệt này.

# Thí dụ minh họa

Trong thí dụ minh họa sau đây, giả định một bác sĩ đang thực hiện kế hoạch phân tích với 3 loại mô hình hồi quy khác nhau:

+ Mô hình hồi quy logistic nhằm khảo sát liên hệ giữa 4 biomarkers A,B,C,D và biến kết quả nhị phân: chẩn đoán Diagnosis,

+ Mô hình hồi quy Gamma nhằm khảo sát liên hệ giữa 4 biomarkers A,B,C,D và một điểm số lâm sàng Score, được mô tả bằng phân phối Gamma.

+ Mô hình hồi quy Gaussian nhằm khảo sát liên hệ giữa 4 biomarkers A,B,C,D và một điểm số lâm sàng Score, mô tả bởi phân phối chuẩn

Các bạn tải dữ liệu từ github của Nhi: 

https://raw.githubusercontent.com/kinokoberuji/Python-snipets/master/GOF/Template_data.csv


In [1]:
warning_status = "ignore"
import warnings
warnings.filterwarnings(warning_status)
with warnings.catch_warnings():
    warnings.filterwarnings(warning_status, category=DeprecationWarning)

import numpy as np
import pandas as pd
from patsy import dmatrices

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import statsmodels.api as sm
import statsmodels

from abc import ABC, abstractmethod

In [2]:
df = pd.read_csv('Template_data.csv', sep = ';', decimal = '.')

df

,Marker_A,Marker_B,Marker_C,Marker_D,Marker_E,Score,Diagnosis
0,31.159420,8.461538,0.000000,27.883212,21.941176,5.378085,0
1,21.739130,10.000000,10.416667,9.598540,6.000000,5.655489,0
2,10.144928,19.230769,10.416667,1.934307,16.752941,5.895959,0
3,15.942029,8.461538,33.333333,18.138686,9.929412,5.407077,0
4,0.000000,0.000000,18.750000,0.802920,9.611765,7.102900,0
...,...,...,...,...,...,...,...
138,38.405797,10.000000,2.083333,5.109489,2.423529,0.509999,0
139,39.130435,19.230769,4.166667,7.481752,8.200000,0.799632,0
140,43.478261,23.076923,6.250000,9.963504,7.035294,3.510305,0
141,45.652174,22.307692,16.666667,27.700730,19.741176,3.030440,0


# Triển khai

Sơ đồ UML của hệ thống như sau:

!['uml'](Template_method_uml.png)

Đầu tiên, Nhi tạo Abstract base class GLM, trong đó có method mặc định là init nhận dữ liệu đầu vào và công thức mô hình Y ~ X, và 3 abstract methods là centering, fit_model và report.

In [3]:
class GLM(ABC):
    
    def __init__(self, data: pd.DataFrame, formula: str):
        
        self.y, self.X = dmatrices(formula_like=formula,
                                   data=data, 
                                   return_type='dataframe')
    
    @abstractmethod
    def centering(self):
        pass
    
    @abstractmethod
    def fit_model(self):
        pass
    
    @abstractmethod
    def report(self):
        pass

Tiếp theo, Nhi tạo 3 subclass tương ứng cho 3 loại mô hình cần thực hiện: chúng kế thừa abstract base class (superclass) GLM, như vậy kế thừa luôn method init nên không cần phải làm bước này. Tuy nhiên 3 methods còn lại là centering, fit_model và report sẽ được tùy chỉnh cho mỗi loại mô hình:


## Subclass Logistic_reg

+ chỉ làm centering cho X

+ fit_model dùng sm.Logit

+ report : tính thêm Odds_ratio bằng hàm np.exp và in 3 bảng kết quả

In [4]:
class Logistic_reg(GLM):
    
    def __repr__(self):
        return f"Subclass mô hình logistic"
        
    def centering(self):
        
        sc = StandardScaler(with_mean=True, with_std=False)
        xmat = sc.fit_transform(self.X.iloc[:,1:])
        self.X.loc[:,1:] = xmat
        print('Trung bình các biến X đã được dịch chuyển về 0')
        
    
    def fit_model(self):
        log_model = sm.Logit(self.y, self.X)
        
        result = log_model.fit()
        rep = result.summary2()
        rep.tables.append(rep.tables[1].iloc[:,[0,4,5]].apply(np.exp))
        rep.tables[2].rename(columns={"Coef.": "Odds-ratio"}, inplace = True)
        
        self.rep = rep
        
        print('Khớp xong  mô hình logistic với dữ liệu')
    
    def report(self):
        
        print(self.rep.tables[0].to_string(index=False, header = False))
        print('='*30)
        print(self.rep.tables[1].to_string())
        print('='*30)
        print(self.rep.tables[2].to_string())

## Subclass Gamma_reg

+ Chỉ làm centering cho X,

+ fit_model bằng sm.GLM, family là Gamma, link_function là log

+ report: exponential cho hệ số hồi quy rồi in 3 bảng kết quả

In [5]:
class Gamma_reg(GLM):
    
    def __repr__(self):
        return f"Subclass mô hình hồi quy Gamma"
        
    def centering(self):
        
        sc = StandardScaler(with_mean=True, with_std=False)
        xmat = sc.fit_transform(self.X.iloc[:,1:])
        self.X.loc[:,1:] = xmat
        print('Trung bình các biến X đã được dịch chuyển về 0')
        
    
    def fit_model(self):
        regmod = sm.GLM(self.y, self.X, family = sm.families.Gamma(link = statsmodels.genmod.families.links.log))
                
        result = regmod.fit()
        
        rep = result.summary2()
        rep.tables.append(rep.tables[1].iloc[:,[0,4,5]].apply(np.exp))
        rep.tables[2].rename(columns={"Coef.": "Exp(coefs)"}, inplace = True)
        
        self.rep = rep
        
        print('Khớp xong  mô hình hồi quy Gamma với dữ liệu')
    
    def report(self):
        
        print(self.rep.tables[0].to_string(index=False, header = False))
        print('='*30)
        print(self.rep.tables[1].to_string())
        print('='*30)
        print(self.rep.tables[2].to_string())

## Subclass Gaussian_reg

+ centering cho cả X và Y

+ fit_model bằng sm.GLM đơn giản

+ report: chỉ in 2 bảng kết quả 

In [6]:
class Gaussian_reg(GLM):
    
    def __repr__(self):
        return f"Subclass mô hình hồi quy tuyến tính"
        
    def centering(self):
        
        sc = StandardScaler(with_mean=True, with_std=False)
        xmat = sc.fit_transform(self.X.iloc[:,1:])
        self.X.loc[:,1:] = xmat
        print('Trung bình các cả X và y đều được dịch chuyển về 0')

    def fit_model(self):
        regmod = sm.GLM(self.y, self.X)
                
        result = regmod.fit()
        
        rep = result.summary2()
        
        self.rep = rep
        
        print('Khớp xong  mô hình hồi quy tuyến tính với dữ liệu')
    
    def report(self):
        
        print(self.rep.tables[0].to_string(index=False, header = False))
        print('='*30)
        print(self.rep.tables[1].to_string())

# Khi sử dụng trên thực tế

## Mô hình logistic

Đầu tiên, ta dựng mô hình logistic bằng Logistic_reg subclass:

In [7]:
log_mod = Logistic_reg(data = df, formula = 'Diagnosis ~ Marker_A + Marker_B + Marker_C + Marker_D')

log_mod

Subclass mô hình logistic

In [8]:
log_mod.centering()

Trung bình các biến X đã được dịch chuyển về 0


In [9]:
log_mod.fit_model()

Optimization terminated successfully.
         Current function value: 0.474346
         Iterations 6
Khớp xong  mô hình logistic với dữ liệu


In [10]:
log_mod.report()

             Model:            Logit Pseudo R-squared:      0.307
Dependent Variable:        Diagnosis              AIC:   145.6629
              Date: 2021-04-20 18:33              BIC:   160.4772
  No. Observations:              143   Log-Likelihood:    -67.831
          Df Model:                4          LL-Null:    -97.854
      Df Residuals:              138      LLR p-value: 2.8381e-12
         Converged:           1.0000            Scale:     1.0000
    No. Iterations:           6.0000                             
              Coef.  Std.Err.         z         P>|z|    [0.025    0.975]
Intercept  0.364817  0.213674  1.707356  8.775594e-02 -0.053976  0.783609
Marker_A  -0.081588  0.022415 -3.639908  2.727356e-04 -0.125520 -0.037656
Marker_B   0.147219  0.029388  5.009542  5.455972e-07  0.089620  0.204818
Marker_C   0.044832  0.022495  1.992952  4.626667e-02  0.000742  0.088921
Marker_D   0.055212  0.025259  2.185829  2.882814e-02  0.005705  0.104719
           Odds-ratio    [0.

## Mô hình Gamma

Tiếp theo, ta dựng mô hình hồi quy Gamma bằng Gamma_reg subclass

In [11]:
gam_mod = Gamma_reg(data = df, formula = 'Score ~ Marker_A + Marker_B + Marker_C + Marker_D')

gam_mod

Subclass mô hình hồi quy Gamma

In [12]:
gam_mod.centering()

Trung bình các biến X đã được dịch chuyển về 0


In [13]:
gam_mod.fit_model()

Khớp xong  mô hình hồi quy Gamma với dữ liệu


In [14]:
gam_mod.report()

             Model:              GLM            AIC: 1292.9745
     Link Function:              log            BIC: -571.0844
Dependent Variable:            Score Log-Likelihood:   -641.49
              Date: 2021-04-20 18:34        LL-Null:   -682.00
  No. Observations:              143       Deviance:    113.79
          Df Model:                4   Pearson chi2:      87.9
      Df Residuals:              138          Scale:   0.63673
            Method:             IRLS                          
              Coef.  Std.Err.          z         P>|z|    [0.025    0.975]
Intercept  3.513291  0.066728  52.650923  0.000000e+00  3.382507  3.644076
Marker_A  -0.035416  0.006413  -5.522931  3.333910e-08 -0.047984 -0.022848
Marker_B   0.057170  0.007126   8.022218  1.038528e-15  0.043202  0.071137
Marker_C   0.021226  0.006674   3.180342  1.471011e-03  0.008145  0.034307
Marker_D   0.021876  0.007441   2.940043  3.281665e-03  0.007292  0.036459
           Exp(coefs)     [0.025     0.975]
In

## Mô hình hồi quy tuyến tính Gaussian

Cuối cùng, là mô hình hồi quy tuyến tính bình thường, bằng subclass Gaussian_reg:

In [15]:
norm_mod = Gaussian_reg(data = df, formula = 'Score ~ Marker_A + Marker_B + Marker_C + Marker_D')

norm_mod

Subclass mô hình hồi quy tuyến tính

In [16]:
norm_mod.centering()

Trung bình các cả X và y đều được dịch chuyển về 0


In [17]:
norm_mod.fit_model()

Khớp xong  mô hình hồi quy tuyến tính với dữ liệu


In [18]:
norm_mod.report()

             Model:              GLM            AIC:  1304.8901
     Link Function:         identity            BIC: 71007.7234
Dependent Variable:            Score Log-Likelihood:    -647.45
              Date: 2021-04-20 18:34        LL-Null:    -700.78
  No. Observations:              143       Deviance:     71693.
          Df Model:                4   Pearson chi2:   7.17e+04
      Df Residuals:              138          Scale:     519.51
            Method:             IRLS                           
               Coef.  Std.Err.          z         P>|z|     [0.025     0.975]
Intercept  40.191847  1.906029  21.086688  1.053868e-98  36.456098  43.927596
Marker_A   -0.977967  0.183169  -5.339148  9.338447e-08  -1.336972  -0.618962
Marker_B    1.558876  0.203561   7.658039  1.887929e-14   1.159904   1.957848
Marker_C    0.519045  0.190641   2.722636  6.476334e-03   0.145396   0.892694
Marker_D    0.654523  0.212533   3.079625  2.072616e-03   0.237965   1.071081


# Ưu điểm và nhược điểm

## Ưu điểm

+ Template Method đơn giản, dễ triển khai,

+ Cho phép người dùng tùy biến linh động một phần trong quy trình/giải thuật, để thích nghi với điều kiện áp dụng mới.

+ Ngay cả khi đã thiết kế xong các concrete subclass, và nhận ra còn tồn tại những phần code trùng lặp, ta vẫn có thể cắt chúng và dán vào superclass, để tinh giản nội dung code của chương trình. 

## Nhược điểm: 

+ Người dùng có thể không hay biết về sự tồn tại của Template method (Superclass) nên không khai thác được hết tính năng này để tạo ra subclass mới cho riêng họ, mà chỉ dùng những subclass có sẵn.

+ Khi có nhiều công đoạn và nhiều phiên bản subclass, template method trở nên khó bảo trì

Ghi chú: Liên hệ với Factory Method ta đã học ở bài trước:

Factory Method có thể xem như 1 trường hợp đặc biệt của Template Method; và có thể được dùng như 1 bước trong một Template Method lớn.